In [4]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [15]:
def alignImages(im1, im2):
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)   #将图片转换为灰度图
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()   #调用SIFT算法
    keypoints1, descriptors1 = sift.detectAndCompute(im1Gray, None)   #检测特征点和描述符
    keypoints2, descriptors2 = sift.detectAndCompute(im2Gray, None)
    matcher = cv2.BFMatcher()   #调用Brute Force匹配算法
    matches = matcher.match(descriptors1, descriptors2, None)   #对两张指纹图像进行匹配
    matches.sort(key=lambda x: x.distance, reverse=False)   #将匹配分数进行排序
    numGoodMatches = int(len(matches)*0.2)   #选取排序得分前20%的指纹
    matches = matches[:numGoodMatches]
    #绘制匹配点
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite("matches.jpg", imMatches)   #保存匹配图像
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
    for i, match in enumerate(matches):   #提取匹配点的位置
        points1[i, :] = keypoints1[match.queryIdx].pt 
        points2[i, :] = keypoints2[match.trainIdx].pt  
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)   #寻找仿射矩阵
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))   #对匹配图像进行对齐   
    return im1Reg, h

In [16]:
refFilename = "0403_00032_0001_1_S.bmp"   
imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)   #读取图像
imFilename = "0403_00032_0002_2_S.bmp"
im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
imReg, h = alignImages(im, imReference)   #对齐图像
outFilename = "aligned.bmp"   #存储对齐图像的名称
cv2.imwrite(outFilename, imReg)   #保存对齐图像
print("Estimated homography : \n",  h)   #输出单适应矩阵

Estimated homography : 
 [[1.00931699e+00 4.90469078e-03 8.30080159e+00]
 [1.05283668e-02 1.03959144e+00 4.24211212e+01]
 [4.26509656e-05 5.74836317e-05 1.00000000e+00]]


In [24]:
#读取数据集路径
dir_path = r'D:\Project_code\fingerPrint\CASIA_Fingerprint_Subject_Ageing\2013\uru4000\2' 
folder_list = os.listdir(dir_path)   #查看该路径下文件夹信息
#得到每个文件夹的具体路径
imagePath_list = [os.path.join(dir_path, imageName) for imageName in folder_list] 
imageAlignedPath = []
for i in imagePath_list:
    firstIndex = 0
    imageName_list = os.listdir(i)   #遍历每个文件夹下的图片名称
    for j in range(len(imageName_list)):
        if j == firstIndex:           
            continue
        elif j%5==0:                 
            firstIndex+=5
        else:
            image1Path_list = [os.path.join(i, imageName_list[firstIndex])]  
            imageAlignedPath.append(image1Path_list)        
            image2Path_list = [os.path.join(i, imageName_list[j])]    
            #准备待对齐的指纹路径，格式为前面一张为指纹对齐参考图像，后一张为待对齐图像，不断循环
            imageAlignedPath.append(image2Path_list)   

In [35]:
i = 0 
name = 1
while i < len(imageAlignedPath):
    reference = cv2.imread(imageAlignedPath[i][0], cv2.IMREAD_COLOR)   #读取指纹对齐参考图像  
    aligned = cv2.imread(imageAlignedPath[i+1][0], cv2.IMREAD_COLOR)   #读取待对齐的图像
    aligned_out, h = alignImages(reference, aligned)   #对齐图像
    reference_outName = imageAlignedPath[i][0][-23:-4]   #读取指纹对齐参考图像文件名
    aligned_outName = imageAlignedPath[i+1][0][-23:-4]   #读取指纹待对齐图像的文件名
    #参考图像和对齐图像的存储路径
    save_path1 = r'D:\Project_code\fingerPrint\dataset\Genuine\(6)unaligned_souce\reference'
    save_path2 = r'D:\Project_code\fingerPrint\dataset\Genuine\(6)unaligned_souce\aligned'
    print("Reference image : ", reference_outName)
    print("Saving aligned image : ", aligned_outName) 
    reference_savePath = save_path1 + "\\" + str(name) +'.jpg'
    aligned_savePath = save_path2 + "\\" + str(name) +'.jpg'
    cv2.imwrite(reference_savePath,reference)   #存储指纹对齐参考图像
    cv2.imwrite(aligned_savePath,aligned_out)   #存储已对齐的指纹图像
    i+=2
    name+=1

Reference image :  0403_00001_0000_0_S
Saving aligned image :  0403_00001_0001_1_S
Reference image :  0403_00001_0000_0_S
Saving aligned image :  0403_00001_0002_2_S
Reference image :  0403_00001_0000_0_S
Saving aligned image :  0403_00001_0003_3_S
Reference image :  0403_00001_0000_0_S
Saving aligned image :  0403_00001_0004_4_S
Reference image :  0403_00002_0000_0_S
Saving aligned image :  0403_00002_0001_1_S
Reference image :  0403_00002_0000_0_S
Saving aligned image :  0403_00002_0002_2_S
Reference image :  0403_00002_0000_0_S
Saving aligned image :  0403_00002_0003_3_S
Reference image :  0403_00002_0000_0_S
Saving aligned image :  0403_00002_0004_4_S
Reference image :  0403_00006_0000_0_S
Saving aligned image :  0403_00006_0001_1_S
Reference image :  0403_00006_0000_0_S
Saving aligned image :  0403_00006_0002_2_S
Reference image :  0403_00006_0000_0_S
Saving aligned image :  0403_00006_0003_3_S
Reference image :  0403_00006_0000_0_S
Saving aligned image :  0403_00006_0004_4_S
Refe